# The Injury Risk Predictor

This script predicts the injury risk of MLB players based on their game logs and injury history using a pre-trained XGBoost model.
It includes the following steps:

1. Load game logs and injury data from a database.
2. Preprocess the data to create features for the model.
3. Load the pre-trained model.
4. Predict injury risk probabilities for current players.

## Imports 

In [12]:
import sys
import os
import config
import joblib
import pandas as pd
import numpy as np
import sqlite3
from datetime import timedelta
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE 
from xgboost import plot_importance

# Add the project root directory to Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
sys.path.append(project_root)

# Confirm paths
print("DB_PATH:", config.DB_PATH)
print("INJURY_MODEL_PATH:", config.INJURY_MODEL_PATH)

db_path = config.DB_PATH
model_path = config.INJURY_MODEL_PATH

# Connect to the database
conn = sqlite3.connect(db_path)
cursor = conn.cursor()


DB_PATH: /Users/daniellarson/Desktop/Code/Projects/dodgers_injtrkr/mlb/mlb_players.db
INJURY_MODEL_PATH: /Users/daniellarson/Desktop/Code/Projects/dodgers_injtrkr/mlb/models/injury_risk_model.pkl


## Create Game Logs Df

In [5]:
mlb_player_stats = pd.read_sql_query("SELECT * FROM mlb_player_stats", conn)
mlb_player_stats.columns

Index(['mlb_player_id', 'stat_type', 'stat_group', 'season', 'game_date',
       'team_id', 'team_name', 'opponent_id', 'opponent_name', 'position',
       'gamesPlayed', 'games', 'gamesStarted', 'assists', 'putOuts', 'errors',
       'chances', 'fielding', 'doublePlays', 'triplePlays', 'throwingErrors',
       'rangeFactorPerGame', 'rangeFactorPer9Inn', 'innings', 'inningsPitched',
       'catcherERA', 'flyOuts', 'groundOuts', 'airOuts', 'passedBall', 'wins',
       'losses', 'wildPitches', 'pickoffs', 'runs', 'doubles', 'triples',
       'homeRuns', 'strikeOuts', 'baseOnBalls', 'intentionalWalks', 'hits',
       'hitByPitch', 'avg', 'atBats', 'obp', 'slg', 'ops', 'caughtStealing',
       'stolenBases', 'stolenBasePercentage', 'groundIntoDoublePlay',
       'groundIntoTriplePlay', 'numberOfPitches', 'plateAppearances',
       'totalBases', 'rbi', 'leftOnBase', 'sacBunts', 'sacFlies', 'babip',
       'groundOutsToAirouts', 'catchersInterference', 'atBatsPerHomeRun',
       'summary', '

mlb_player_stats.columns

In [6]:

# Game Logs for all players in the MLB 

query = """
SELECT mlb_player_stats.* , mlb_player_info.*
FROM mlb_player_stats
INNER JOIN mlb_player_info ON mlb_player_stats.mlb_player_id = mlb_player_info.mlb_player_id
WHERE stat_type = 'gameLog'
"""

game_log_pre = pd.read_sql_query(query, conn)

game_log_pre = game_log_pre.loc[:, ~game_log_pre.columns.duplicated()]

selected_columns = [
    # Identity and basic game info
    'fullName', 'mlb_player_id', 'game_date', 'team_name', 'opponent_name',

    # Workload & fatigue
    'gamesPlayed', 'gamesStarted', 'innings', 'plateAppearances', 'numberOfPitches',
    'game_number', 'atBatsPerHomeRun',

    # Performance (extended)
    'atBats', 'runs', 'hits', 'totalBases', 'doubles', 'triples', 'homeRuns', 'rbi',
    'baseOnBalls', 'intentionalWalks', 'strikeOuts', 'stolenBases', 'caughtStealing',
    'hitByPitch', 'sacBunts', 'sacFlies', 'avg', 'obp', 'slg',

    # Performance trends / condition indicators
    #'groundIntoDoublePlay', 'leftOnBase',

    # Positional & biomechanical stress factors
     'primaryPosition', 'pitchHand', 'batSide',

    # Biometric and career timeline
    'height', 'weight', 'currentAge', 'birthDate', 'debutDate'
]

game_logs_df = game_log_pre[selected_columns]

# Fill rows with missing values
game_logs_df = game_logs_df.fillna(0)

game_logs_df = game_logs_df.rename(columns={
    # Identity and team info
    'fullName': 'Name',
    'mlb_player_id': 'PlayerID',
    'game_date': 'Date',
    'team_name': 'Team',
    'opponent_name': 'OPP',

    # Workload & fatigue
    'gamesPlayed': 'GP',
    'gamesStarted': 'GS',
    'innings': 'INN',
    'plateAppearances': 'PA',
    'numberOfPitches': 'NP',
    'game_number': 'Game#',
    'atBatsPerHomeRun': 'AB/HR',

    # Batting performance
    'atBats': 'AB',
    'runs': 'R',
    'hits': 'H',
    'totalBases': 'TB',
    'doubles': '2B',
    'triples': '3B',
    'homeRuns': 'HR',
    'rbi': 'RBI',
    'baseOnBalls': 'BB',
    'intentionalWalks': 'IBB',
    'strikeOuts': 'SO',
    'stolenBases': 'SB',
    'caughtStealing': 'CS',
    'hitByPitch': 'HBP',
    'sacBunts': 'SAC',
    'sacFlies': 'SF',
    'avg': 'AVG',
    'obp': 'OBP',
    'slg': 'SLG',

    # Performance trends
    'groundIntoDoublePlay': 'GIDP',
    'leftOnBase': 'LOB',

    # Positional & biomechanics
    'position': 'POS',
    'primaryPosition': 'PrimaryPOS',
    'pitchHand': 'Throw',
    'batSide': 'Bat',

    # Biometric & career data
    'height': 'Height',
    'weight': 'Weight',
    'currentAge': 'Age',
    'birthDate': 'BirthDate',
    'debutDate': 'Debut'
})

# STEP 1: Load data
game_logs_df.rename(columns={"PlayerID": "mlb_player_id"}, inplace=True)
game_logs = game_logs_df # must include 'mlb_player_id' and 'Date'
injury_stats = pd.read_sql_query('SELECT * from injury_stats', conn)  # must include 'mlb_player_id' and 'injury_date'

# STEP 2: Convert date columns
game_logs["Date"] = pd.to_datetime(game_logs["Date"])
injury_stats["injury_date"] = pd.to_datetime(injury_stats["injury_date"])

# STEP 3: Initialize all as not injured
game_logs["Injured"] = 0

# STEP 4: Iterate over each injury record
for _, row in injury_stats.iterrows():
    player_id = row["mlb_player_id"]
    injury_date = row["injury_date"]

    # Mark all games within 30 days before the injury as Injured = 1
    mask = (
        (game_logs["mlb_player_id"] == player_id) &
        (game_logs["Date"] <= injury_date) &
        (game_logs["Date"] >= injury_date - timedelta(days=30))
    )
    game_logs.loc[mask, "Injured"] = 1

game_logs.rename(columns={"mlb_player_id": "PlayerID"}, inplace=True)



In [7]:
game_logs.head(5)

,Name,PlayerID,Date,Team,OPP,GP,GS,INN,PA,NP,...,SLG,PrimaryPOS,Throw,Bat,Height,Weight,Age,BirthDate,Debut,Injured
0,Donovan Solano,456781,2024-05-06,San Diego Padres,Chicago Cubs,1,0.0,0,4.0,20.0,...,.667,First Base,Right,Right,"5' 8""",210,37,1987-12-17,2012-05-21,0
1,Donovan Solano,456781,2024-05-07,San Diego Padres,Chicago Cubs,1,0.0,0,4.0,15.0,...,.429,First Base,Right,Right,"5' 8""",210,37,1987-12-17,2012-05-21,0
2,Donovan Solano,456781,2024-05-11,San Diego Padres,Los Angeles Dodgers,1,0.0,0,4.0,26.0,...,.500,First Base,Right,Right,"5' 8""",210,37,1987-12-17,2012-05-21,0
3,Donovan Solano,456781,2024-05-15,San Diego Padres,Colorado Rockies,1,0.0,0,4.0,18.0,...,.538,First Base,Right,Right,"5' 8""",210,37,1987-12-17,2012-05-21,0
4,Donovan Solano,456781,2024-05-19,San Diego Padres,Atlanta Braves,1,0.0,0,0.0,0.0,...,.538,First Base,Right,Right,"5' 8""",210,37,1987-12-17,2012-05-21,0


## Model Training

In [102]:
injury_risk_model = '/Users/daniellarson/Desktop/Code/Projects/dodgers_injtrkr/mlb/models/injury_risk_model.pkl'

# Load trained model and feature names
xgb_model = joblib.load(injury_risk_model)
feature_names = xgb_model.get_booster().feature_names

# STEP 1: Load current player data (replace this with your data source)
# Example: new_game_logs = pd.read_csv('game_logs.csv')

new_game_logs = game_logs.copy()

# STEP 2: Preprocess data (same as training)
new_game_logs["Date"] = pd.to_datetime(new_game_logs["Date"])
new_game_logs = new_game_logs.drop(columns=["Name", "Date", "Team", "OPP"])

# Identify stat columns
stats_cols = new_game_logs.drop(columns=["PlayerID", "Injured"]).columns

new_game_logs_numeric = new_game_logs[stats_cols].apply(pd.to_numeric, errors='coerce')

# Prepare rolling DataFrames
roll5 = new_game_logs_numeric.groupby(new_game_logs["PlayerID"]).rolling(5, min_periods=1).mean().reset_index(level=0, drop=True).add_suffix('_roll5')
roll10 = new_game_logs_numeric.groupby(new_game_logs["PlayerID"]).rolling(10, min_periods=1).mean().reset_index(level=0, drop=True).add_suffix('_roll10')

# Concatenate once to avoid fragmentation
new_game_logs = pd.concat([new_game_logs, roll5, roll10], axis=1)

# Drop raw stats and player ID
new_game_logs = new_game_logs.drop(columns=list(stats_cols) + ["PlayerID"])
new_game_logs = new_game_logs.fillna(0)

# STEP 3: Filter for active, uninjured players
uninjured_players = new_game_logs[new_game_logs['Injured'] == 0]

# Drop label before prediction and ensure feature order matches training
X_uninjured = uninjured_players.drop(columns=['Injured'])
X_uninjured = X_uninjured[feature_names]

# STEP 4: Predict injury risk probabilities
injury_probs = xgb_model.predict_proba(X_uninjured)[:, 1]

# STEP 5: Add probabilities to DataFrame
uninjured_players = uninjured_players.copy()
uninjured_players['Injury_Risk'] = injury_probs

# STEP 6: Add back PlayerID and Name for clarity (from original game_logs)
uninjured_players['PlayerID'] = game_logs.loc[uninjured_players.index, 'PlayerID']
uninjured_players['Name'] = game_logs.loc[uninjured_players.index, 'Name']

# STEP 7: Rank players by predicted injury risk (highest to lowest)
ranked_risk = uninjured_players[['PlayerID', 'Name', 'Injury_Risk']]
ranked_risk = (
    uninjured_players[['PlayerID', 'Name', 'Injury_Risk']]
    .sort_values(by='Injury_Risk', ascending=False)
    .drop_duplicates(subset='PlayerID', keep='first')
)
# Add a formatted percentage column
ranked_risk['Injury_Risk_Percent'] = (ranked_risk['Injury_Risk'] * 100).round(2).astype(str) + '%'

# Add a rank column
ranked_risk['Rank'] = ranked_risk['Injury_Risk'].rank(method='first', ascending=False).astype(int)
ranked_risk = ranked_risk[['Rank', 'Name', 'PlayerID', 'Injury_Risk_Percent']]

## Top 30 MLB Injury List

In [103]:
# Top 30 current mlb players with highest injury risk

ranked_risk.head(30)

,Rank,Name,PlayerID,Injury_Risk_Percent
15672,1,Kyle Tucker,663656,98.69%
30757,2,Bryson Stott,681082,98.51%
10618,3,Willy Adames,642715,98.44%
5208,4,Jeimer Candelario,600869,98.37%
30164,5,Steven Kwan,680757,98.23%
24304,6,Jared Triolo,669707,98.13%
17759,7,Griffin Conine,665052,98.06%
386086,8,Dustin Harris,687957,98.06%
6969,9,Travis Jankowski,608671,97.87%
17212,10,Alec Bohm,664761,97.87%


In this analysis, we used an XGBoost classification model trained on MLB player game logs, incorporating rolling averages (over the last 5 and 10 games) of key workload, performance, and biometric features. For each active, uninjured player, the model predicts an injury risk probability between 0 and 1, which we convert into an easy-to-read percentage (e.g., 98.69%).

The Injury_Risk score reflects how closely a player’s recent workload and stats match patterns historically observed before injuries (within 30 days prior) in the training data. For example, high-risk players like Kyle Tucker and Bryson Stott show recent game trends (heavy workloads, high batting/fielding activity) that the model has learned to associate with elevated injury likelihood.

The ranking you see is sorted by predicted risk, with the top players showing metrics (e.g., high plate appearances, frequent starts, or accumulated fatigue) that the model interprets as increasing their odds of injury. Keep in mind, the precision and recall balance in the model is tuned to prioritize catching as many potential injuries as possible (high recall), even if it means some false positives (lower precision), which explains why even elite or healthy players might appear near the top.

## LA Dodgers Injury Risk

In [107]:
# Injury risk prediction for the Los Angeles Dodgers in 2025

dodgers_game_logs = game_logs[
    (game_logs['Team'] == 'Los Angeles Dodgers') & (game_logs['Date'].dt.year == 2025)
]

# Load trained model and feature names
injury_risk_model = '/Users/daniellarson/Desktop/Code/Projects/dodgers_injtrkr/mlb/models/injury_risk_model.pkl'
xgb_model = joblib.load(injury_risk_model)
feature_names = xgb_model.get_booster().feature_names

# Preprocess data
dodgers_new_game_logs = dodgers_game_logs.copy()
dodgers_new_game_logs = dodgers_new_game_logs.drop(columns=["Name", "Date", "Team", "OPP"])
dodgers_stats_cols = dodgers_new_game_logs.drop(columns=["PlayerID", "Injured"]).columns

# Calculate rolling averages
dodgers_new_game_logs_numeric = dodgers_new_game_logs[dodgers_stats_cols].apply(pd.to_numeric, errors='coerce')
dodgers_roll5 = (
    dodgers_new_game_logs_numeric.groupby(dodgers_new_game_logs["PlayerID"])
    .rolling(5, min_periods=1).mean().reset_index(level=0, drop=True).add_suffix('_roll5')
)
dodgers_roll10 = (
    dodgers_new_game_logs_numeric.groupby(dodgers_new_game_logs["PlayerID"])
    .rolling(10, min_periods=1).mean().reset_index(level=0, drop=True).add_suffix('_roll10')
)

# Combine once to avoid fragmentation
dodgers_new_game_logs = pd.concat([dodgers_new_game_logs, dodgers_roll5, dodgers_roll10], axis=1)
dodgers_new_game_logs = dodgers_new_game_logs.drop(columns=list(dodgers_stats_cols) + ["PlayerID"])
dodgers_new_game_logs = dodgers_new_game_logs.fillna(0)

# Filter active, uninjured players
dodgers_uninjured_players = dodgers_new_game_logs[dodgers_new_game_logs['Injured'] == 0]

# Prepare data for prediction
X_dodgers_uninjured = dodgers_uninjured_players.drop(columns=['Injured'])
X_dodgers_uninjured = X_dodgers_uninjured[feature_names]

# Predict injury risk
dodgers_injury_probs = xgb_model.predict_proba(X_dodgers_uninjured)[:, 1]

# Add predictions
dodgers_uninjured_players = dodgers_uninjured_players.copy()
dodgers_uninjured_players['Injury_Risk'] = dodgers_injury_probs

# Add back PlayerID and Name
dodgers_uninjured_players['PlayerID'] = game_logs.loc[dodgers_uninjured_players.index, 'PlayerID']
dodgers_uninjured_players['Name'] = game_logs.loc[dodgers_uninjured_players.index, 'Name']

# Rank players by risk (keep only highest per player)
dodgers_ranked_risk = (
    dodgers_uninjured_players[['PlayerID', 'Name', 'Injury_Risk']]
    .sort_values(by='Injury_Risk', ascending=False)
    .drop_duplicates(subset=['PlayerID'], keep='first')
    .reset_index(drop=True)
)
# Add a formatted percentage column
dodgers_ranked_risk['Injury_Risk_Percent'] = (dodgers_ranked_risk['Injury_Risk'] * 100).round(2).astype(str) + '%'


In [108]:
#top 10 current dodgers players with highest injury risk

print(dodgers_ranked_risk.head(10))

   PlayerID               Name  Injury_Risk Injury_Risk_Percent
0    571970          Max Muncy     0.929502              92.95%
1    687221     Dalton Rushing     0.895328              89.53%
2    681546       James Outman     0.846650              84.66%
3    808975       Hyeseong Kim     0.819206              81.92%
4    621035       Chris Taylor     0.809580              80.96%
5    592696      Eddie Rosario     0.798012               79.8%
6    606192  Teoscar Hernández     0.781091              78.11%
7    518692    Freddie Freeman     0.770524              77.05%
8    676439    Hunter Feduccia     0.646990               64.7%
9    605141       Mookie Betts     0.643853              64.39%


## Phillies Injury risk 

In [89]:
# Injury risk prediction for the Philadelphia Phillies in 2025

phillies_game_logs = game_logs[
    (game_logs['Team'] == 'Philadelphia Phillies') & (game_logs['Date'].dt.year == 2025)
]

phillies_new_game_logs = phillies_game_logs.copy()
phillies_new_game_logs = phillies_new_game_logs.drop(columns=["Name", "Date", "Team", "OPP"])
phillies_stats_cols = phillies_new_game_logs.drop(columns=["PlayerID", "Injured"]).columns

# Calculate rolling averages
phillies_new_game_logs_numeric = phillies_new_game_logs[phillies_stats_cols].apply(pd.to_numeric, errors='coerce')
phillies_roll5 = (
    phillies_new_game_logs_numeric.groupby(phillies_new_game_logs["PlayerID"])
    .rolling(5, min_periods=1).mean().reset_index(level=0, drop=True).add_suffix('_roll5')
)
phillies_roll10 = (
    phillies_new_game_logs_numeric.groupby(phillies_new_game_logs["PlayerID"])
    .rolling(10, min_periods=1).mean().reset_index(level=0, drop=True).add_suffix('_roll10')
)

# Combine once to avoid fragmentation
phillies_new_game_logs = pd.concat([phillies_new_game_logs, phillies_roll5, phillies_roll10], axis=1)
phillies_new_game_logs = phillies_new_game_logs.drop(columns=list(phillies_stats_cols) + ["PlayerID"])
phillies_new_game_logs = phillies_new_game_logs.fillna(0)

# Filter active, uninjured players
phillies_uninjured_players = phillies_new_game_logs[phillies_new_game_logs['Injured'] == 0]

# Prepare data for prediction
X_phillies_uninjured = phillies_uninjured_players.drop(columns=['Injured'])
X_phillies_uninjured = X_phillies_uninjured[feature_names]

# Predict injury risk
phillies_injury_probs = xgb_model.predict_proba(X_phillies_uninjured)[:, 1]

# Add predictions
phillies_uninjured_players = phillies_uninjured_players.copy()
phillies_uninjured_players['Injury_Risk'] = phillies_injury_probs

# Add back PlayerID and Name
phillies_uninjured_players['PlayerID'] = game_logs.loc[phillies_uninjured_players.index, 'PlayerID']
phillies_uninjured_players['Name'] = game_logs.loc[phillies_uninjured_players.index, 'Name']

# Rank players by risk (keep only highest per player)
phillies_ranked_risk = (
    phillies_uninjured_players[['PlayerID', 'Name', 'Injury_Risk']]
    .sort_values(by='Injury_Risk', ascending=False)
    .drop_duplicates(subset=['PlayerID'], keep='first')
    .reset_index(drop=True)
)
phillies_ranked_risk['Injury_Risk_Percent'] = (phillies_ranked_risk['Injury_Risk'] * 100).round(2).astype(str) + '%'

#top 10 players with highest injury risk

print(phillies_ranked_risk.head(10))

   PlayerID              Name  Injury_Risk Injury_Risk_Percent
0    664761         Alec Bohm     0.974924              97.49%
1    607208       Trea Turner     0.915824              91.58%
2    669016     Brandon Marsh     0.830791              83.08%
3    665561    Rafael Marchán     0.825344              82.53%
4    624641      Edmundo Sosa     0.777536              77.75%
5    592663     J.T. Realmuto     0.613901              61.39%
6    592206  Nick Castellanos     0.604436              60.44%
7    596146        Max Kepler     0.601463              60.15%
8    679032       Johan Rojas     0.587693              58.77%
9    681082      Bryson Stott     0.565465              56.55%


## Risk Prediction for custom Team and Season

In [106]:
# Choose team and season
selected_team = 'Los Angeles Angels'  # <-- change this to any MLB team
selected_year = 2025 # <-- change this to any year

# Filter game logs for the selected team and season
team_game_logs = game_logs[
    (game_logs['Team'] == selected_team) & (game_logs['Date'].dt.year == selected_year)
]

# Preprocess data
team_new_game_logs = team_game_logs.copy()
team_new_game_logs = team_new_game_logs.drop(columns=["Name", "Date", "Team", "OPP"])
team_stats_cols = team_new_game_logs.drop(columns=["PlayerID", "Injured"]).columns

# Calculate rolling averages
team_new_game_logs_numeric = team_new_game_logs[team_stats_cols].apply(pd.to_numeric, errors='coerce')
team_roll5 = (
    team_new_game_logs_numeric.groupby(team_new_game_logs["PlayerID"])
    .rolling(5, min_periods=1).mean().reset_index(level=0, drop=True).add_suffix('_roll5')
)
team_roll10 = (
    team_new_game_logs_numeric.groupby(team_new_game_logs["PlayerID"])
    .rolling(10, min_periods=1).mean().reset_index(level=0, drop=True).add_suffix('_roll10')
)

# Combine once to avoid fragmentation
team_new_game_logs = pd.concat([team_new_game_logs, team_roll5, team_roll10], axis=1)
team_new_game_logs = team_new_game_logs.drop(columns=list(team_stats_cols) + ["PlayerID"])
team_new_game_logs = team_new_game_logs.fillna(0)

# Filter active, uninjured players
team_uninjured_players = team_new_game_logs[team_new_game_logs['Injured'] == 0]

# Prepare data for prediction
X_team_uninjured = team_uninjured_players.drop(columns=['Injured'])
X_team_uninjured = X_team_uninjured[feature_names]

# Predict injury risk
team_injury_probs = xgb_model.predict_proba(X_team_uninjured)[:, 1]

# Add predictions
team_uninjured_players = team_uninjured_players.copy()
team_uninjured_players['Injury_Risk'] = team_injury_probs

# Add back PlayerID and Name
team_uninjured_players['PlayerID'] = game_logs.loc[team_uninjured_players.index, 'PlayerID']
team_uninjured_players['Name'] = game_logs.loc[team_uninjured_players.index, 'Name']

# Rank players by risk (keep only highest per player)
team_ranked_risk = (
    team_uninjured_players[['PlayerID', 'Name', 'Injury_Risk']]
    .sort_values(by='Injury_Risk', ascending=False)
    .drop_duplicates(subset=['PlayerID'], keep='first')
    .reset_index(drop=True)
)
team_ranked_risk['Injury_Risk_Percent'] = (team_ranked_risk['Injury_Risk'] * 100).round(2).astype(str) + '%'


# Add a rank column
team_ranked_risk['Rank'] = team_ranked_risk['Injury_Risk'].rank(method='first', ascending=False).astype(int)
team_ranked_risk = team_ranked_risk[['Rank', 'Name', 'PlayerID', 'Injury_Risk_Percent']]

# Show top 10 players with highest injury risk
print(f"Top 10 Injury Risk Predictions for {selected_team} in {selected_year}:")
team_ranked_risk.head(10)

Top 10 Injury Risk Predictions for Los Angeles Angels in 2025:


,Rank,Name,PlayerID,Injury_Risk_Percent
0,1,Jorge Soler,624585,96.43%
1,2,Taylor Ward,621493,95.02%
2,3,Luis Rengifo,650859,84.55%
3,4,Jo Adell,666176,77.82%
4,5,Travis d'Arnaud,518595,76.34%
5,6,Tim Anderson,641313,75.16%
6,7,Kyren Paris,677347,73.13%
7,8,Mike Trout,545361,71.57%
8,9,Nolan Schanuel,694384,66.15%
9,10,Logan O'Hoppe,681351,58.8%
